In [16]:
!pip install kaggle
# subir json de kaggle
from google.colab import files
files.upload()

# configurar api
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# descargar el set al path...
!kaggle datasets download -d ambityga/imagenet100 -p /content/imagenet100 --unzip
# quedarme solo con las carpeta y archivo que importan y borrar el resto
!mv /content/imagenet100/val.X /content/val.X
!mv /content/imagenet100/Labels.json /content/Labels.json
!rm -rf /content/imagenet100



Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/ambityga/imagenet100
License(s): unknown
100% 16.1G/16.1G [04:13<00:00, 95.0MB/s]
100% 16.1G/16.1G [04:13<00:00, 68.2MB/s]


In [34]:
import json

# cargar labels
with open('/content/Labels.json') as f:
    labels_dict = json.load(f)

# mapping de nombre de carpetas a índices
class_to_idx = {key: idx for idx, key in enumerate(labels_dict.keys())}

# ej para ver mapping
print(class_to_idx)



{'n01968897': 0, 'n01770081': 1, 'n01818515': 2, 'n02011460': 3, 'n01496331': 4, 'n01847000': 5, 'n01687978': 6, 'n01740131': 7, 'n01537544': 8, 'n01491361': 9, 'n02007558': 10, 'n01735189': 11, 'n01630670': 12, 'n01440764': 13, 'n01819313': 14, 'n02002556': 15, 'n01667778': 16, 'n01755581': 17, 'n01924916': 18, 'n01751748': 19, 'n01984695': 20, 'n01729977': 21, 'n01614925': 22, 'n01608432': 23, 'n01443537': 24, 'n01770393': 25, 'n01855672': 26, 'n01560419': 27, 'n01592084': 28, 'n01914609': 29, 'n01582220': 30, 'n01667114': 31, 'n01985128': 32, 'n01820546': 33, 'n01773797': 34, 'n02006656': 35, 'n01986214': 36, 'n01484850': 37, 'n01749939': 38, 'n01828970': 39, 'n02018795': 40, 'n01695060': 41, 'n01729322': 42, 'n01677366': 43, 'n01734418': 44, 'n01843383': 45, 'n01806143': 46, 'n01773549': 47, 'n01775062': 48, 'n01728572': 49, 'n01601694': 50, 'n01978287': 51, 'n01930112': 52, 'n01739381': 53, 'n01883070': 54, 'n01774384': 55, 'n02037110': 56, 'n01795545': 57, 'n02027492': 58, 'n0153

In [35]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ImageNetValDataset(Dataset):
    def __init__(self, root_dir, labels_dict, class_to_idx, transform=None):
        self.root_dir = root_dir
        self.labels_dict = labels_dict
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for folder_name, label in labels_dict.items():
            folder_path = os.path.join(root_dir, folder_name)
            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(self.class_to_idx[folder_name])  # label num

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]  # label num

        if self.transform:
            image = self.transform(image)

        return image, label



In [38]:
from torch.utils.data import DataLoader

# transformada del set
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# cargar el set
val_dataset = ImageNetValDataset('/content/val.X', labels_dict, class_to_idx, transform=transform_val)


# dataloader
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [28]:
import torch
import torchvision.models as models
# definir modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50_weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=resnet50_weights)
model = model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [41]:
import torch.nn.functional as F

# 'class_to_idx' mapea las 100 clases a sus índices (0-999 del ImageNet entero)
idx_to_class = {v: k for k, v in class_to_idx.items()}  # Reverse mapping para indices -> class names

correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        # restringir modelo a  las 100 clases
        # crear tensor que se queda solo con los output logits relevantes (que corresponden a las 100 clases)
        outputs_relevant = torch.zeros(outputs.size(0), len(class_to_idx)).to(device)

        for i, class_name in enumerate(class_to_idx.keys()):
            class_idx = class_to_idx[class_name]  # index del imagenet completo
            outputs_relevant[:, i] = outputs[:, class_idx]  # logits relevantes

        # predicción para las 100 clases
        _, predicted = torch.max(outputs_relevant, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')




Accuracy: 1.14%
